# pip

In [1]:
!pip install tensorflow_addons

     |████████████████████████████████| 706kB 6.9MB/s 


# Function

In [2]:
### F1 score for metrics ###
# import keras.backend as K

# def f1_score(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     recall = true_positives / (possible_positives + K.epsilon())
#     f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
#     return f1_val



### learning rate schedule ###
# def exponetial_decay(lr0, s):
#   def exponetial_decay_fn(epoch):
#     return lr0 * 0.1 ** (epoch / s)
#   return exponetial_decay_fn


### F1 score for callback ###
# from sklearn.metrics import f1_score
# from keras.callbacks import Callback

# class F1Callback(Callback):
  
#   def __init__(self, model, x_val, y_val):
#     self.model = model
#     self.x_val = x_val
#     self.y_val = y_val

#   def on_epoch_end(self, epoch, logs):
#     pred = self.model.predict(self.x_val)
#     f1 = f1_score(self.y_val, np.round(pred))
#     print('f1_val: {0}'.format(f1))

# Static

In [3]:
TRAIN_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/train_images.csv'
TEST_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/test_images.csv'
SAMPLE_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/sample_submit.csv'

TRAIN_IMAGE_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/train_images'
TEST_IMAGE_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/test_images'

MODEL_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/model.h5'

TRAIN_TEST_SPLIT = 0.2
IMAGE_SIZE = 640
BATCH_SIZE = 4
EPOCHS = 100

# CONV_DROPOUT_RATE = 0.2

# Import data

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

df = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

df['id'] = TRAIN_IMAGE_PATH+'/'+df['id']

df['class_num'] = df['class_num'].astype(str)

print('df shape: {0}, df_test shape: {1}'.format(df.shape, df_test.shape))

df shape: (1102, 2), df_test shape: (1651, 1)


# ImageDataGenerator

In [5]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size=TRAIN_TEST_SPLIT, random_state=2021)

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(featurewise_center=False,
                                   samplewise_center=False,
                                   featurewise_std_normalization=False,
                                   samplewise_std_normalization=False,
                                   zca_whitening=False,
                                   rotation_range=180,
                                   width_shift_range=1.0,
                                   height_shift_range=1.0,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=0.0,
                                   fill_mode='nearest',
                                   cval=0.0,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rescale=1./255.,
                                   preprocessing_function=None,
                                   data_format=None,)
# train_datagen = ImageDataGenerator(rescale=1./255.)
val_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_dataframe(df_train, directory=TRAIN_IMAGE_PATH, x_col='id', y_col='class_num', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, class_mode='categorical')
val_generator = val_datagen.flow_from_dataframe(df_val, directory=TRAIN_IMAGE_PATH, x_col='id', y_col='class_num', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, class_mode='categorical')
# train_generator = train_datagen.flow_from_dataframe(df, directory=TRAIN_IMAGE_PATH, x_col='id', y_col='class_num', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, class_mode='sparse')

for data_batch, labels_batch in train_generator:
  print('data batch shape: ', data_batch.shape)
  print('labels batch shape: ', labels_batch.shape)
  break

Found 881 validated image filenames belonging to 4 classes.
Found 221 validated image filenames belonging to 4 classes.
data batch shape:  (4, 640, 640, 3)
labels batch shape:  (4, 4)


# Fine tuning

In [6]:
from keras import models
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB5, VGG16, ResNet50
from keras.layers import Conv2D, Activation, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten

model = models.Sequential()

conv_base = VGG16(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)
# conv_base = ResNet50(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)
# conv_base = EfficientNetB5(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)

### VGG16 ###
# TARGET_LAYER = 'blcok4_conv3'
# TARGET_LAYER = 'block5_conv1'
# TARGET_LAYER = 'block5_conv2'
# TARGET_LAYER = 'block5_conv3'
### ResNet50 ###
# TARGET_LAYER = 'conv5_block3_1_conv'
# TARGET_LAYER = 'conv5_block1_1_conv'
# TARGET_LAYER = 'conv4_block1_1_conv'

# conv_base.trainable = True
# set_trainable = False

# for layer in conv_base.layers:
#   if layer.name == TARGET_LAYER:
#     set_trainable = True
#   if set_trainable:
#     layer.trainable = True
#   else:
#     layer.trainable = False

### Transfer learing ###
for layer in conv_base.layers:
  layer.trainable = False

model.add(conv_base)

# model.add(Flatten())
model.add(GlobalAveragePooling2D())
# model.add(BatchNormalization())
# model.add(Dense(256, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dense(256, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dense(256, activation='relu'))
# model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# conv_base.summary()
model.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 20, 20, 512)       14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 2048)              1050624   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________

# Create model



In [9]:
from keras.optimizers import Adam, SGD
from tensorflow_addons.losses import WeightedKappaLoss
from tensorflow_addons.metrics import F1Score, CohenKappa

# model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(1e-5), metrics=['accuracy'])
# model.compile(loss='sparse_categorical_crossentropy', optimizer=SGD(lr=1e-5, momentum=0.9, nesterov=True), metrics=['accuracy'])
# model.compile(loss=WeightedKappaLoss(num_classes=4), optimizer=SGD(lr=1e-5, momentum=0.9, nesterov=True), metrics=['accuracy'])
# model.compile(loss='sparse_categorical_crossentropy', optimizer=SGD(lr=1e-5, momentum=0.9, nesterov=True), metrics=['accuracy', CohenKappa(num_classes=4)])
# model.compile(loss=WeightedKappaLoss(num_classes=4), optimizer=SGD(lr=1e-5, momentum=0.9, nesterov=True), metrics=['accuracy', F1Score(num_classes=4), CohenKappa(num_classes=4)])

### Transfer learning ###
# model.compile(loss='sparse_categorical_crossentropy', optimizer=SGD(lr=0.2, momentum=0.9, decay=0.01), metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-2), metrics=['accuracy'])

# Make callbacks and Training

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import time

MONITOR_EARLYSTOPPING = 'val_loss'
MONITOR_MODELCHECKPOINT = 'val_loss'

callbacks_list = [EarlyStopping(monitor=MONITOR_EARLYSTOPPING, patience=5, mode='min',),
                  # LearningRateScheduler(exponetial_decay(lr0=1e-2, s=10)),
                  ModelCheckpoint(filepath=MODEL_PATH, monitor=MONITOR_MODELCHECKPOINT, save_best_only=True, mode='min',),]

history = model.fit(train_generator, epochs=EPOCHS, callbacks=EarlyStopping(monitor=MONITOR_EARLYSTOPPING, patience=1, mode='min',), validation_data=val_generator)

### Transfer learning ###
for layer in conv_base.layers:
  layer.trainable = True

# model.compile(loss='sparse_categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, decay=0.001), metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-5), metrics=['accuracy'])

start = time.time()

history = model.fit(train_generator, epochs=EPOCHS, callbacks=callbacks_list, validation_data=val_generator)

elapsed_time = time.time() - start
print('Elapsed time: {:.1f}[min]'.format(elapsed_time / 60))

Epoch 1/100
221/221 [==============================] - 657s 3s/step - loss: 5.6037 - accuracy: 0.2484 - val_loss: 1.3893 - val_accuracy: 0.3258
Epoch 2/100
221/221 [==============================] - 85s 386ms/step - loss: 1.3917 - accuracy: 0.3630 - val_loss: 1.3871 - val_accuracy: 0.3258
Epoch 3/100
221/221 [==============================] - 84s 378ms/step - loss: 1.3553 - accuracy: 0.3662 - val_loss: 1.3768 - val_accuracy: 0.3258
Epoch 4/100
221/221 [==============================] - 84s 378ms/step - loss: 1.3751 - accuracy: 0.3325 - val_loss: 1.3894 - val_accuracy: 0.3258
Epoch 1/100
221/221 [==============================] - 102s 454ms/step - loss: 1.3764 - accuracy: 0.3341 - val_loss: 1.3892 - val_accuracy: 0.3258
Epoch 2/100
221/221 [==============================] - 104s 468ms/step - loss: 1.3618 - accuracy: 0.3569 - val_loss: 1.3890 - val_accuracy: 0.3258
Epoch 3/100
221/221 [==============================] - 104s 468ms/step - loss: 1.3616 - accuracy: 0.3568 - val_loss: 1.3889 

# Evaluate

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

fig = plt.figure(0)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'b', color='orange', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

fig.savefig('acc.jpg')

plt.show()

fig = plt.figure(1)

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'b', color='orange', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

fig.savefig('loss.jpg')

plt.show()

# Submit

In [ ]:
from keras.models import load_model
import numpy as np
from PIL import Image

clf = load_model(MODEL_PATH)
# clf = load_model(MODEL_PATH, custom_objects={'f1_score': f1_score})

preds = []

for image_id in df_test['id']:
    image = Image.open(TEST_IMAGE_PATH+'/'+image_id)
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    image = np.expand_dims(image, axis=0)
    image = image / 255.
    preds.append(np.argmax(clf.predict(image)))
    # preds.append(np.argmax(model.predict(image)))

df_test['label'] = preds

df_test.to_csv('/content/drive/MyDrive/SANDBOX/DATASET/submission.csv', index=False, header=False)

df_test.head()

# As for Google Colab

```javascript
function ClickConnect(){ 
console.log("Working"); 
document.querySelector("#comments > span").click()
}
setInterval(ClickConnect,500000)
```